In [1]:
#NOTE: use paimg1 env, the retccl one has package issue with torchvision
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path
from scipy.spatial.distance import cdist


sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, count_label
from Utils import log_message, set_seed
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles, get_feature_label_array_dynamic
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
SELECTED_FEATURE = [str(i) for i in range(0,2048)] + ['TUMOR_PIXEL_PERC']
TUMOR_FRAC_THRES = 0
TRAIN_SAMPLE_SIZE = "ALLTUMORTILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0
SELECTED_FOLD = 0
CLUSTER_ALG = 'KMEAN'
N_CLUSTERS = 4
CLUSTER_DIST = 'L2'

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
wsi_path = proj_dir + '/data/OPX/'
label_path = proj_dir + 'data/MutationCalls/'
train_tile_path = proj_dir + 'intermediate_data/cancer_prediction_results110224/IMSIZE250_OL' + str(TRAIN_OVERLAP) + '/'
test_tile_path =  proj_dir + 'intermediate_data/cancer_prediction_results110224/IMSIZE250_OL' + str(TEST_OVERLAP) + '/'
feature_name = 'features_alltiles_retccl'

indata_path = proj_dir + 'intermediate_data/model_ready_data/' +'MAX_SS'+ str(TRAIN_SAMPLE_SIZE)  + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + "/split_fold" + str(SELECTED_FOLD) + "/" 
data_info_path = indata_path + "init_data_info/"
updated_feature_path =indata_path + "clusters/" + CLUSTER_ALG + "/"
updated_feature_folder_name = "_NCLUSTER_" + str(N_CLUSTERS) +  "_DISTMETRIC_" + CLUSTER_DIST 

################################################
#Create output dir
################################################
outdir =  indata_path + ""
create_dir_if_not_exists(outdir0)
outdir =   os.path.join(outpath, "clusters", CLUSTER_ALG)
create_dir_if_not_exists(outdir)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/MAX_SSALLTUMORTILES_TrainOL100_TestOL0/split_fold0//clusters/' created.
cuda:0


In [4]:
############################################################################################################
#Get features and labels
############################################################################################################
train_feature = torch.load(updated_feature_path + 'updated_train_feature' + updated_feature_folder_name + '.pth')
test_feature = torch.load(updated_feature_path + 'updated_test_feature' + updated_feature_folder_name + '.pth')
val_feature = torch.load(updated_feature_path + 'updated_val_feature' + updated_feature_folder_name+ '.pth')

train_label = torch.load(data_info_path + 'train_label.pth')
test_label = torch.load(data_info_path + 'test_label.pth')
val_label = torch.load(data_info_path + 'val_label.pth')

train_ids = torch.load(data_info_path + 'train_ids.pth')
test_ids = torch.load(data_info_path + 'test_ids.pth')
val_ids = torch.load(data_info_path + 'val_ids.pth')

In [35]:
# import umap
# from sklearn.preprocessing import StandardScaler
# reducer = umap.UMAP()
# scaled = StandardScaler().fit_transform(cluster_feature)
# embedding = reducer.fit_transform(scaled)

In [29]:
# Plot the data points and cluster centers with cluster labels
plt.figure(figsize=(10, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c=cluster_labels_train, cmap='viridis',alpha=0.6, s = 0.001)
for i, center in enumerate(centers):
    plt.scatter(center[0], center[1], c='red', marker=f'${i}$', s=200)  # Use cluster label as marker

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('K-Means Clustering for Tile Embeddings on PCs')
plt.grid(True)
plt.savefig(outdir  + 'original_cluster_scatter.png')
plt.close()

In [98]:
#Get Cluster labels
cluster_labels_test = get_cluster_label(test_feature_df, centers, feature_for_cluster)
cluster_labels_val = get_cluster_label(valid_feature_df, centers, feature_for_cluster)

#add cluster label to df
train_feature_df['Cluster'] = cluster_labels_train
test_feature_df['Cluster'] = cluster_labels_test
valid_feature_df['Cluster'] = cluster_labels_val

#Get updated feature
updated_feature_names= list(range(0,2048)) + ['TUMOR_PIXEL_PERC','Cluster']
updated_train_feature = get_updated_feature(train_feature_df, train_ids, updated_feature_names)
updated_test_feature = get_updated_feature(test_feature_df, test_ids, updated_feature_names)
updated_val_feature = get_updated_feature(valid_feature_df, val_ids, updated_feature_names)


torch.save(updated_train_feature, outdir2 + 'updated_train_feature.pth')
torch.save(updated_test_feature,  outdir2 + 'updated_test_feature.pth')
torch.save(updated_val_feature,   outdir2 + 'updated_val_feature.pth')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
0
10
20
30
0


(4426, 2050)

In [ ]:
################################################
#     Model ready data 
################################################
train_data = ModelReadyData_diffdim(updated_train_feature,train_label,train_tf_info)
test_data = ModelReadyData_diffdim(updated_test_feature,test_label,test_tf_info)
val_data = ModelReadyData_diffdim(updated_val_feature,val_label,val_tf_info)

#Output
torch.save(train_data, outdir2 + 'train_data.pth')
torch.save(test_data,  outdir2 + 'test_data.pth')
torch.save(val_data,   outdir2 + 'val_data.pth')

torch.save(train_info,   outdir2 + 'train_info.pth')
torch.save(test_info,   outdir2 + 'test_info.pth')
torch.save(val_info,   outdir2 + 'val_info.pth')

torch.save(train_tf_info,   outdir2 + 'train_tf_info.pth')
torch.save(test_tf_info,   outdir2 + 'test_tf_info.pth')
torch.save(val_tf_info,   outdir2 + 'val_tf_info.pth')


torch.save(train_ids,   outdir2 + 'train_ids.pth')
torch.save(test_ids,   outdir2 + 'test_ids.pth')
torch.save(val_ids,   outdir2 + 'val_ids.pth')

In [ ]:
#Cluster anlaysis
#Assign cluster
train_feature_df['PC1'] = principal_components[:, 0]
train_feature_df['PC2'] = principal_components[:, 1]

In [ ]:
#Plot scatter cluster plot for each outcome,
for plot_outcome in SELECTED_LABEL:
    plot_data = train_feature_df[['PC1','PC2', 'Cluster'] + [plot_outcome]]
        
    # Create scatter plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Separate dots by Cluster but color by Outcome
    for cluster in plot_data['Cluster'].unique():
        subset = plot_data[plot_data['Cluster'] == cluster]
        ax.scatter(subset['PC1'], subset['PC2'], 
                   s=np.where(subset[plot_outcome] == 1, 20, 0.001), 
                   c=['steelblue' if outcome == 0 else 'darkred' for outcome in subset[plot_outcome]], 
                   alpha=0.6,
                   linewidth=1.5, label=f'Cluster {cluster}',
                   zorder=3 if (subset[plot_outcome] == 1).any() else 2)
    
    
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_title(plot_outcome)
    plt.grid(True)
    plt.savefig(outdir1 +  'cluster_scatter_' + plot_outcome + '.png')
    plt.close()

In [ ]:
# Plot distribution of outcome by cluster (stacked bar plot)   
for plot_outcome in SELECTED_LABEL:
    plot_data = train_feature_df[['PC1','PC2', 'Cluster'] + [plot_outcome]]
 
    # Create a crosstab to count the occurrences of each outcome per cluster
    crosstab = pd.crosstab(plot_data['Cluster'], plot_data[plot_outcome])
    
    # Calculate the percentage of each outcome per cluster
    percentage_crosstab = crosstab.div(crosstab.sum(axis=1), axis=0) * 100
    
    # Plot the stacked bar chart
    percentage_crosstab.plot(kind='bar', stacked=True, color=['steelblue', 'darkred'])
    plt.xlabel('Cluster')
    plt.ylabel('Percentage')
    plt.title('Bar Chart of ' + plot_outcome + ' per Cluster')
    plt.legend(title=plot_outcome, loc='center left', bbox_to_anchor=(1.0, 0.5))
    
    plt.tight_layout()
    plt.savefig(outdir1 + "outcome_distribution_" + plot_outcome + '.png')
    plt.close()

In [ ]:
# Plot combined distribution of outcome by cluster 
for plot_outcome in SELECTED_LABEL:
    plot_data = train_feature_df[['PC1','PC2', 'Cluster'] + [plot_outcome]]

    # Create a crosstab to count the occurrences of each outcome per cluster
    crosstab = pd.crosstab(plot_data[plot_outcome],plot_data['Cluster'])

    # Calculate the percentage of each outcome per cluster
    percentage_crosstab = crosstab.div(crosstab.sum(axis=1), axis=0) * 100
    
    # Plot the stacked bar chart
    percentage_crosstab.plot(kind='bar', stacked=False, color=['#440154','#3b528b','#5ec962','#fde725'])
    plt.xlabel(plot_outcome)
    plt.ylabel('Percentage')
    plt.title('Bar Chart of Clusters Per Outcome')
    plt.legend(title='Cluster', loc='center left', bbox_to_anchor=(1.0, 0.5))

    plt.tight_layout()
    plt.savefig(outdir1  + 'cluster_distribution_' +  plot_outcome + '.png')
    plt.close()

In [ ]:
save_image_size = 250
pixel_overlap = 100
mag_extract = 20
limit_bounds = True
TOP_K = 5
pretrain_model_name = "retccl"
mag_target_prob = 2.5
smooth = False
mag_target_tiss = 1.25

In [ ]:
i = 6
pt = selected_train_ids[i]
cur_df = train_feature_df.loc[train_feature_df['ID'] == pt,['ID','Cluster']]
cur_info_df = train_info[i]

print(["Mutation labels  :","AR","HR","PTEN","RB1","TP53","TMB","MSI_POS"])
print(train_label[i])

_file = wsi_path + pt + ".tif"
oslide = openslide.OpenSlide(_file)
save_name = str(Path(os.path.basename(_file)).with_suffix(''))


#Generate tiles
tiles, tile_lvls, physSize, base_mag = generate_deepzoom_tiles(oslide,save_image_size, pixel_overlap, limit_bounds)

#get level 0 size in px
l0_w = oslide.level_dimensions[0][0]
l0_h = oslide.level_dimensions[0][1]

#1.25x tissue detection for mask
from Utils import get_downsample_factor, get_image_at_target_mag
from Utils import do_mask_original,check_tissue,whitespace_check
import cv2
if 'OPX' in pt:
    rad_tissue = 5
elif '(2017-0133)' in pt:
    rad_tissue = 2
lvl_resize_tissue = get_downsample_factor(base_mag,target_magnification = mag_target_tiss) #downsample factor
lvl_img = get_image_at_target_mag(oslide,l0_w, l0_h,lvl_resize_tissue)
tissue, he_mask = do_mask_original(lvl_img, lvl_resize_tissue, rad = rad_tissue)

cur_comb_df = pd.concat([cur_info_df, cur_df], axis = 1)

#2.5x for probability maps
lvl_resize = get_downsample_factor(base_mag,target_magnification = mag_target_prob) #downsample factor
x_map = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
x_count = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)

for index, row in cur_comb_df.iterrows():
    cur_xy = row['TILE_XY_INDEXES'].strip("()").split(", ")
    x ,y = int(cur_xy[0]) , int(cur_xy[1])
    
    #Extract tile for prediction
    lvl_in_deepzoom = tile_lvls.index(mag_extract)
    tile_starts, tile_ends, save_coords, tile_coords = extract_tile_start_end_coords(tiles, lvl_in_deepzoom, x, y) #get tile coords
    map_xstart, map_xend, map_ystart, map_yend = get_map_startend(tile_starts,tile_ends,lvl_resize) #Get current tile position in map

    #Store predicted probabily in map and count
    try: 
        x_count[map_xstart:map_xend,map_ystart:map_yend] += 1
        x_map[map_xstart:map_xend,map_ystart:map_yend] += row['Cluster']
    except:
        pass

print('post-processing')
x_count = np.where(x_count < 1, 1, x_count)
x_map = x_map / x_count

x_sm = x_map

he_mask = cv2.resize(np.uint8(he_mask),(x_sm.shape[1],x_sm.shape[0])) #resize to output image size
#TODO:
#get cancer_mask:
# cancer_mask == 
# x_sm[(he_mask == 1) & (x_sm == 0)] = 0.1 #If tissue map value > 1, then x_sm = 1
x_sm[he_mask < 1] = -1 

plt.imshow(x_sm, cmap='Spectral_r')
plt.colorbar()
plt.savefig(os.path.join(outdir1, save_name + '_cluster.png'), dpi=500,bbox_inches='tight')
plt.show()
plt.close()